1. Choose derised probability distribution $P$
2. Compute acceptance matrix $$A(i,j) = min(1,\dfrac{P(j)}{P(i)})$$
3. Compute transition matrix $$T(i,j) = g(i,j) A(i,j) , i \neq j$$
$$T(i,j) = g(i,j) A(i,j) + \sum_{k \neq i} g(i,k)(1-A(i,k)) , i = j$$
where we choose $g$ is uniform distribution, i.e $g(i \rightarrow j) = 1/N$ for all $i,j$. 

4. Use the transition matrix, compute equilibrium probaility $P^{eq}$ to check if it consistents with derised probability..

In [2]:
import numpy as np 
import matplotlib.pyplot as plt

In [61]:
#generate acceptance probability matrix
def acceptance_matrix(P,N):
    P_acc = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if (P[i]==0):
                P_acc[i][j] = 1
                
            else:
                P_acc[i][j] = min(1,P[j]/P[i])
    return P_acc

In [63]:
P = np.array([0.5,0.2,0.05,0.05,0.2]) #given a derised probability    
N=5 #number of states
P_acc = acceptance_matrix(P,N)
print(P_acc)  

[[1.   0.4  0.1  0.1  0.4 ]
 [1.   1.   0.25 0.25 1.  ]
 [1.   1.   1.   1.   1.  ]
 [1.   1.   1.   1.   1.  ]
 [1.   1.   0.25 0.25 1.  ]]


# Using transition matrix

In [64]:
#compute transition matrix from acceptance matrix
def transition_matrix(P_acc,N):
    #compute transition matrix
    T = np.zeros((N,N))
    T=1/5*P_acc
    for i in range(N):
        for k in range(N):
            if (i!=k):
                T[i][i] +=1/5*(1-P_acc[i][k])
    return T

In [65]:
#print transition matrix  
transition_matrix(P_acc,5)   

array([[0.8 , 0.08, 0.02, 0.02, 0.08],
       [0.2 , 0.5 , 0.05, 0.05, 0.2 ],
       [0.2 , 0.2 , 0.2 , 0.2 , 0.2 ],
       [0.2 , 0.2 , 0.2 , 0.2 , 0.2 ],
       [0.2 , 0.2 , 0.05, 0.05, 0.5 ]])

In [66]:
# check the correctness of transition matrix, follow all step in tutorial 3
def probability(init_prob,trans_mat,num_chain=64):
    if (num_chain ==0):
        next_prob = init_prob
    else:
        for i in range(num_chain):
            next_prob = init_prob.dot(trans_mat)
            # print(next_prob)
            init_prob = next_prob
    return init_prob

init_prob = np.array([1/2,1/2,0,0,0])
trans_mat = transition_matrix(P_acc,5)
probability(init_prob,trans_mat)

array([0.5 , 0.2 , 0.05, 0.05, 0.2 ])

# second way using Markov chain

1. Compute acceptance matrix $$A(i,j) = min(1,\dfrac{P(j)}{P(i)})$$
2. Generate an initial random state $s_i$
3. Generate a proposed state $s_j$ with probability $p$.
4. Compare $p$ and $A(i \rightarrow j)$:
+ $p<= A(i \rightarrow j)$ : accept $s_j$
+ $p > A(i \rightarrow j)$ : reject, keep same state $s_i$
5. Repeat this proceture and compute observale.

In [67]:
#generate a random state
def random_state():
    n = np.random.uniform(0,1)
    if (n < 0.2):
        s= 0
    elif (0.2 <= n <0.4):
        s= 1 
    elif (0.4 <= n <0.6):
        s= 2
    elif (0.6 <= n <0.8):
        s= 3
    else:
        s= 4
    return s

In [68]:
# compute observable after n-steps
def markov_chain(P_acc,iterations =1000000):
    #generate an initial random_state
    s = random_state()
    o = 0 #observale
    acc = 0 #number of acceptance
    oe =0 #observale's error
    for iter in range(iterations):       
        #generate a next state
        sp = random_state()
        n = np.random.uniform(0,1)
        if (n < P_acc[s][sp]):
            acc +=1
            s = sp
        o += s
        oe += s**2
        
        if (iter%100000 ==0)&(iter!=0): 
            print("iteration=",iter)
            print("acceptance=",acc/(iter+1))
            print("observable=",o/(iter+1),"+-",np.sqrt(((iter+1)*oe-o**2)/(iter+1))/(iter+1))
            print('-'*18)
markov_chain(P_acc)

iteration= 100000
acceptance= 0.5809341906580934
observable= 1.2496075039249608 +- 0.004993691617214648
------------------
iteration= 200000
acceptance= 0.5830470847645762
observable= 1.2594887025564871 +- 0.0035326833404708447
------------------
iteration= 300000
acceptance= 0.582374725417582
observable= 1.2577424741917527 +- 0.0028844113336702905
------------------
iteration= 400000
acceptance= 0.581281046797383
observable= 1.2575068562328595 +- 0.002497979219990956
------------------
iteration= 500000
acceptance= 0.5811468377063246
observable= 1.2535614928770142 +- 0.002231540490866
------------------
iteration= 600000
acceptance= 0.5808656985571691
observable= 1.2530195783007028 +- 0.002037783115097025
------------------
iteration= 700000
acceptance= 0.5810577413460838
observable= 1.2540953512923554 +- 0.0018870951851708696
------------------
iteration= 800000
acceptance= 0.5813767732790334
observable= 1.2554059307425867 +- 0.0017656807269933173
------------------
iteration= 900000